In [2]:
# https://github.com/chanzuckerberg/single-cell-curation/issues/516
# https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/4.0.0/schema.md#feature_length

import numpy as np
import os
import scanpy as sc
import subprocess

In [2]:
def validate(file):
    validate_process = subprocess.run(['cellxgene-schema', 'validate', '--add-labels', file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in validate_process.stdout.decode('utf-8').split('\n'):
        print(line)
    for line in validate_process.stderr.decode('utf-8').split('\n'):
        print(line)
        if 'is_valid=' in line:
            valid = line.split('=')[-1]
            return valid

In [3]:
def save_and_test(adata, expected):
    if ('feature_length' in adata.var.columns) and ('feature_biotype' in adata.var.columns):
        print(adata.var[['feature_biotype','feature_length']].info())
        print('-------------------')
        print(adata.var[['feature_length','feature_biotype']].value_counts())
        
    elif 'feature_biotype' in adata.var.columns: 
        print('feature_length is absent')
    
    else: 
        print('feature_biotype is absent')

    print('------------------')

    adata.write(filename='test.h5ad')
    valid = validate('test.h5ad')
    print('------------------')
    if expected != valid:
        print('\033[1m\033[91mERROR\033[0m')
    else:
        print('\033[1m\033[92mPASSED\033[0m')
    os.remove('test.h5ad')

In [8]:
adata = sc.read_h5ad('../output_valid.h5ad') #backed='r' would be slightly quicker but produces an error with multiple writes

In [9]:
adata.var

,Unnamed: 0,feature_is_filtered,feature_name,feature_reference,feature_biotype
feature_id,,,,,
ENSMUSG00000029422,7173.0,False,Rsrc2,NCBITaxon:10090,gene
ENSMUSG00000049036,18484.0,False,Tmem121,NCBITaxon:10090,gene
ENSMUSG00000029577,7022.0,False,Ube3b,NCBITaxon:10090,gene
ENSMUSG00000040746,15950.0,False,Rnf167,NCBITaxon:10090,gene
ENSMUSG00000020590,17963.0,False,Snx13,NCBITaxon:10090,gene
...,...,...,...,...,...
ENSMUSG00000102368,792.0,False,4930590L20Rik,NCBITaxon:10090,gene
ENSMUSG00000021033,18266.0,False,Gstz1,NCBITaxon:10090,gene
ENSMUSG00000030657,10469.0,False,Xylt1,NCBITaxon:10090,gene


In [10]:
adata.raw.var

,Unnamed: 0,feature_name,feature_reference,feature_biotype
feature_id,,,,
ENSMUSG00000029422,7173.0,Rsrc2,NCBITaxon:10090,gene
ENSMUSG00000049036,18484.0,Tmem121,NCBITaxon:10090,gene
ENSMUSG00000029577,7022.0,Ube3b,NCBITaxon:10090,gene
ENSMUSG00000040746,15950.0,Rnf167,NCBITaxon:10090,gene
ENSMUSG00000020590,17963.0,Snx13,NCBITaxon:10090,gene
...,...,...,...,...
ENSMUSG00000102368,792.0,4930590L20Rik,NCBITaxon:10090,gene
ENSMUSG00000021033,18266.0,Gstz1,NCBITaxon:10090,gene
ENSMUSG00000030657,10469.0,Xylt1,NCBITaxon:10090,gene


**Test valid cases**

**Test invalid cases**